# CNN MNIST

In [1]:
#importing functions from python3 to python2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

#importing numpy and tensorflow
import numpy as np
import tensorflow as tf

#ignore all the warnings and don't show them in the notebook
import warnings
warnings.filterwarnings('ignore')

#treshold on what messages are to be logged
tf.logging.set_verbosity(tf.logging.INFO)
#importing debug library
from tensorflow.python import debug as tf_debug

## Debugger

### Uncomment the below line and execute the code to run the debugger.

### Go to the link once you start execution    			http://localhost:6006/

In [ ]:
#Uncomment the below line to run the debugger
#Add monitor=[hook] as a parameter to the estimators below
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")

In [5]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
            inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
      }
    #if predict mode, run the estimator and return the values
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
def main(unused_argv):
    # Load training data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    #load the the train data labels
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    #load the test data
    eval_data = mnist.test.images  # Returns np.array
    #load the test data labels
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)


In [7]:
#Run the model, steps set to 200 instead of 20000 as the execution time was large
#Changing steps back to 20000 in model training results in an accuracy of 97%
if __name__ == "__main__":
    tf.app.run()


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9df93637f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
I

INFO:tensorflow:loss = 2.2696, step = 201
INFO:tensorflow:probabilities = [[ 0.11421116  0.0837108   0.1137443   0.09868434  0.09403597  0.09169564
   0.09631579  0.08846508  0.1084328   0.11070412]
 [ 0.09148124  0.10830866  0.09369586  0.09761549  0.10898986  0.10060444
   0.09967624  0.09939343  0.0947429   0.10549188]
 [ 0.10360489  0.11275978  0.11554828  0.08917973  0.10964888  0.09174568
   0.10317016  0.08834971  0.09007908  0.09591381]
 [ 0.08995516  0.10379037  0.11936557  0.09334806  0.11024822  0.08649674
   0.11055771  0.08317003  0.10193864  0.10112956]
 [ 0.09655234  0.10640556  0.09510366  0.09758613  0.12011177  0.0818587
   0.09145747  0.09910982  0.10568109  0.1061335 ]
 [ 0.09682651  0.08910979  0.111701    0.10206254  0.10948984  0.10169201
   0.09822597  0.08861541  0.09730168  0.10497524]
 [ 0.09601466  0.10825725  0.10408176  0.10281243  0.10747497  0.08773994
   0.10543239  0.09998108  0.09606431  0.09214125]
 [ 0.10682639  0.10259867  0.12140163  0.08921432  0

INFO:tensorflow:global_step/sec: 7.33047
INFO:tensorflow:probabilities = [[ 0.11118226  0.08456438  0.11306273  0.1118002   0.0943349   0.08678818
   0.11989574  0.07416063  0.09908403  0.10512705]
 [ 0.10251644  0.09267349  0.1006529   0.09901934  0.10578277  0.09810626
   0.1030343   0.11431428  0.09653137  0.08736887]
 [ 0.10719549  0.08108433  0.10138496  0.09837369  0.1065155   0.09373485
   0.10175441  0.09945221  0.11434616  0.09615844]
 [ 0.10752681  0.08932139  0.09602469  0.09794764  0.11036965  0.09434632
   0.09815876  0.09134728  0.10642546  0.10853194]
 [ 0.10051936  0.08281207  0.09458771  0.10904752  0.11322103  0.08901098
   0.10039219  0.0888714   0.11249957  0.10903817]
 [ 0.11334498  0.09988172  0.10819723  0.1126536   0.10171469  0.09129148
   0.09195009  0.09446271  0.10325523  0.08324829]
 [ 0.11607187  0.09818467  0.10848201  0.10516503  0.10526423  0.08688079
   0.10075811  0.09306882  0.08862762  0.09749684]
 [ 0.09645078  0.11036538  0.10429213  0.10139543  0

INFO:tensorflow:loss = 2.26436, step = 301 (13.642 sec)
INFO:tensorflow:probabilities = [[ 0.10253153  0.10331105  0.11096579  0.10744326  0.09205973  0.07824305
   0.10531968  0.09324414  0.10574395  0.10113779]
 [ 0.10773413  0.11515832  0.11605903  0.09470977  0.09690627  0.09980996
   0.09283667  0.08196031  0.09654595  0.0982796 ]
 [ 0.10574701  0.09318838  0.10326366  0.10401254  0.10161044  0.09768089
   0.11017003  0.09774115  0.09612089  0.09046493]
 [ 0.09268315  0.11072945  0.10300913  0.10012588  0.10690652  0.09501324
   0.08802613  0.10947071  0.09418313  0.09985266]
 [ 0.11109903  0.09608622  0.11100233  0.11947691  0.08358303  0.09851617
   0.10691393  0.0938562   0.090655    0.08881113]
 [ 0.11602408  0.09302349  0.10210775  0.09748169  0.10515678  0.09411167
   0.11947235  0.07892767  0.09879889  0.0948957 ]
 [ 0.10145513  0.09224728  0.10642653  0.09991727  0.10965502  0.09692319
   0.0899209   0.09149864  0.09930995  0.11264615]
 [ 0.09045638  0.09111637  0.10586214

INFO:tensorflow:Saving checkpoints for 400 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 2.22469.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-15-00:54:30
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-15-00:54:34
INFO:tensorflow:Saving dict for global step 400: accuracy = 0.4303, global_step = 400, loss = 2.21531
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 400: /tmp/mnist_convnet_model/model.ckpt-400
{'accuracy': 0.4303, 'loss': 2.2153082, 'global_step': 400}


SystemExit: 